In [2]:
import networkx as nx

from clustice.geometry import tune_layout, constellation
from clustice.gromacs import render
from clustice.topology import ice_graph
from clustice.coder import encode
from clustice.water import tip4p

import py3Dmol

# note: g must be a graph whose labels start from 0.
g = nx.dodecahedral_graph() # dodecahedral 20mer

# O-O distance
L = 0.27

# rough estimate of the positions of the nodes
layout = constellation(g, edgelen=L)

# optimize the tetrahedral arrangements
layout = tune_layout(g, layout, edgelen=L)

# set orientations of the hydrogen bonds.
dg = ice_graph(g)

# get the unique id for the graph
id = encode(dg)
print(id)

# put water molecules
gro = render(dg, layout, watermodel=tip4p)
with open(f'{id}','w') as f:
    f.write(gro)

# show
view = py3Dmol.view()
view.addModel(gro, 'gro')
view.setStyle({'stick':{}})
view.addUnitCell()
view.zoomTo()
view.show()


/Users/matto/.local/share/virtualenvs/ClustIce-TrkXxxIw/lib/python3.11/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


1 0.1629628809399074
2 0.1466440915643435
4 0.14348666200100396
8 0.14339281084903432
16 0.1433928221903437
32 0.1433928258488734
64 0.14339282585583812
1.2.3+6.19.3.4+6.7.14.1+7.8+13.0+9.10+12.16.12.13+15.5.15+17.4+18.11.0+18


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

[(1, 4, 3), (1, 5)]


Rebuild the cluster from id.

In [3]:
import networkx as nx

from clustice.geometry import tune_layout, constellation
from clustice.gromacs import render
from clustice.coder import decode
from clustice.water import tip4p

import py3Dmol

s = "1.2.3+6.19.3.4+6.7.14.1+7.8+13.0+9.10+12.16.12.13+15.5.15+17.4+18.11.0+18"

dg = decode(s)

# Make layout of the nodes
g = nx.Graph(dg)
L = 0.27
layout = constellation(g, edgelen=L)
layout = tune_layout(g, layout, edgelen=L)

# put water molecules
gro = render(dg, layout, watermodel=tip4p)

# show
view = py3Dmol.view()
view.addModel(gro, 'gro')
view.setStyle({'stick':{}})
view.addUnitCell()
view.zoomTo()
view.show()


/Users/matto/.local/share/virtualenvs/ClustIce-TrkXxxIw/lib/python3.11/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


1 0.16296408115423205
2 0.14664472590612057
4 0.1434867829090457
8 0.1433928192657355
16 0.1433928221877597
32 0.14339282584871998
64 0.14339282585583812


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol